In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

### Part 1:
- Split the data into a 70-30 split for training and testing data. 

In [2]:
# open the dataset and see its contents
PATH = '/Users/planetkevin/Assignments/ATMS523/HW5/machine-learning-1-kevinpb2/homework/'
dat = pd.read_csv('radar_parameters.csv', index_col=0)

In [3]:
dat

,Zh (dBZ),Zdr (dB),Ldr (dB),Kdp (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...
18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


The dataset contains radar parameters that we'll consider features. The goal is construct a model that uses these features to effectively predict our target variable, rain rate. 

In [4]:
Xcol = dat.columns[:-1]
Ycol = dat.columns[-1]

X = dat[Xcol]
Y = dat[Ycol]

# use sci-kit learn to seperate our dataset into training and testing datasets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.7)

### Part 2:
- Using the previously created split, train a multiple linear regression model, and validate it using the testing dataset. 
- Compare the $R^{2}$ and $RMSE$ on the training and testing dataset to a prediction of rain rate based on the formula $Z = 200R^{1.6}$

In [5]:
# training the model
model = LinearRegression(fit_intercept=True)
model.fit(Xtrain, Ytrain)

# model fitting on training dataset
Ytrain_fit = model.predict(Xtrain)

In [6]:
# briefly evaluating model performance
print('Model Performance: Training Data')
print('RMSE: ', np.round(mean_squared_error(Ytrain, Ytrain_fit, squared=False),2))
print('R\u00b2: ', np.round(r2_score(Ytrain, Ytrain_fit), 3))

Model Performance: Training Data
RMSE:  0.92
R²:  0.989


In [7]:
# validating with the testing set
Ypred = model.predict(Xtest)

# evaluating skill
print('Model Performance: Testing Data')
print('RMSE: ', np.round(mean_squared_error(Ypred, Ytest, squared=False),2))
print('R\u00b2: ', np.round(r2_score(Ypred, Ytest), 3))

Model Performance: Testing Data
RMSE:  0.94
R²:  0.987


The model is capable of both representing the training dataset and predicting the testing dataset well. 

In [8]:
# producing the predictions based on the empirical relationship
def RR(dBZ):
    """
    The formula for dBZ: dBZ = 10log(z)
    The formula for Z: 200R^{1.6}
    After solving, R is given by: R = 0.036*10^{0.0625Z}
    """
    RR = 0.036*10**(0.0625*dBZ)
    return RR

dat['MP_R (mm/hr)'] = RR(dat['Zh (dBZ)'])

In [9]:
# evaluating skill of the empirical relationship on the testing data
tindices = list(Ytest.index)

print('Performance: Marshall-Palmer Relationship')
print('RMSE: ', np.round(mean_squared_error(Ytest, dat['MP_R (mm/hr)'].loc[tindices], squared=False), 2))
print('R\u00b2: ', np.round(r2_score(Ytest, dat['MP_R (mm/hr)'].loc[tindices]), 3))

Performance: Marshall-Palmer Relationship
RMSE:  7.0
R²:  0.265


It is clear that the MLR model we developed is more skillful at predicting rain rate than the Marshall-Palmer relationship. 

### Part 3: 
- Do a grid search over polynomial orders (from 0-21) and use cross-validation of 7 folds. 
- Choose the best performing polynomial model and evaluate its performance. 
- Compare its performance to prior model formulations. 

In [10]:
# preparing the polynomial model
def PolynomialRegression(degree=0):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(fit_intercept=True))

params = {'polynomialfeatures__degree': np.arange(7)}

grid_poly = GridSearchCV(PolynomialRegression(), params, cv=7)

In [11]:
# fit the model
grid_poly.fit(Xtrain, Ytrain)

/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
7 fits failed out of a total of 49.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_para

GridSearchCV(cv=7,
             estimator=Pipeline(steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=0)),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6])})

In [12]:
# the best performing polynomial model
print(grid_poly.best_params_)
poly_model = grid_poly.best_estimator_

{'polynomialfeatures__degree': 2}


In [13]:
Ypred_poly = poly_model.predict(Xtest)

# evaluate the polynomial model performance
print('Performance: Polynomial Model')
print('RMSE: ', np.round(mean_squared_error(Ypred_poly, Ytest, squared=False),2))
print('R\u00b2: ', np.round(r2_score(Ypred_poly, Ytest), 3))

Performance: Polynomial Model
RMSE:  0.19
R²:  0.999


The best performing iteration of the polynomial model (of order 2) outperforms the multiple linear regression and empirical models. 

### Part 4:
- Repeat the previous step using Random Forest Regression.

In [14]:
# we're perform a grid search on the following parameters
params = {'bootstrap': [True, False],  
          'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],  
          'max_features': ['auto', 'sqrt'],  
          'min_samples_leaf': [1, 2, 4],  
          'min_samples_split': [2, 5, 10],  
          'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

# prepare the model
grid_forest = RandomizedSearchCV(RandomForestRegressor(), params, cv=7, n_jobs=-1, n_iter=100)

In [15]:
grid_forest.fit(Xtrain,Ytrain)

/glade/u/apps/opt/conda/envs/npl/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomizedSearchCV(cv=7, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]})

In [16]:
# the best performing model formulation
print(grid_forest.best_params_)
forest_model = grid_forest.best_estimator_

{'n_estimators': 1800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': True}


In [17]:
Ypred_forest = forest_model.predict(Xtest)

# evaluate the polynomial model performance
print('Performance: Random Forest Regression')
print('RMSE: ', np.round(mean_squared_error(Ypred_forest, Ytest, squared=False),2))
print('R\u00b2: ', np.round(r2_score(Ypred_forest, Ytest), 3))

Performance: Random Forest Regression
RMSE:  0.96
R²:  0.986


The second order polynomial model outperforms the best-performing iteration of the Random Forest regression model, but the Random Forest regression model outperforms the empirical model (baseline) and multiple linear regression model. 